In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv
/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv


In [41]:
data = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')
data.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


This data has no null. So I don't have to remove null

In [43]:
#make nuimeric variables to range variables
def num_convert(num, age) :
    return int(age/num)

data["age"] = data["age"].apply(lambda x : num_convert(10, x))
data["trtbps"] = data["trtbps"].apply(lambda x : num_convert(5, x)) #안정시 혈압 : Steady blood pressure
data["chol"] = data["chol"].apply(lambda x : num_convert(5, x)) #BMI
data["thalachh"] = data["thalachh"].apply(lambda x : num_convert(5, x)) #최대 심박수 : Maximum heart rate

data.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,6,1,3,29,46,1,0,30,0,2.3,0,0,1,1
1,3,1,2,26,50,0,1,37,0,3.5,0,0,2,1
2,4,0,1,26,40,0,0,34,0,1.4,2,0,2,1
3,5,1,1,24,47,0,1,35,0,0.8,2,0,2,1
4,5,0,0,24,70,0,1,32,1,0.6,2,0,2,1


In [53]:
print(len(data))
train_x = data.iloc[10:-10]
train_x = train_x.drop("output", axis = 1)
train_y = data.iloc[10:-10]["output"]

test_x = data.iloc[:10]
test_x = test_x.append(data.iloc[-10:])
test_x = test_x.drop("output", axis = 1)
test_y = data.iloc[:10]["output"]
test_y = test_y.append(data.iloc[-10:]["output"])

print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

train_x.head()

303
(283, 13) (283,)
(20, 13) (20,)


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
10,5,1,0,28,47,0,1,32,0,1.2,2,0,2
11,4,0,2,26,55,0,1,27,0,0.2,2,0,2
12,4,1,1,26,53,0,1,34,0,0.6,2,0,2
13,6,1,3,22,42,0,0,28,1,1.8,1,0,2
14,5,0,3,30,56,1,0,32,0,1.0,2,0,2


In [54]:
#Logistic Regression_sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

lr = LogisticRegression(random_state = 0)
params = {'C' : [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 1, 3, 5, 10], 'penalty' : ['l1', 'l2']}

model = GridSearchCV(lr, params, cv = 5)
model.fit(train_x, train_y)
print('best estimator : ', model.best_estimator_)

model = model.best_estimator_
train_pred = model.predict(train_x)
print('train_accuracy : ', accuracy_score(train_y, train_pred))

test_pred = model.predict(test_x)
print('valid_accuracy : ', accuracy_score(test_y, test_pred))

best estimator :  LogisticRegression(C=0.5, random_state=0)
train_accuracy :  0.8551236749116607
valid_accuracy :  0.8
